In [ ]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import google.generativeai as genai
import os
import re
from dotenv import load_dotenv

load_dotenv('.env.local')

class MultiIntentPhoneSystem:
    def __init__(self, json_file_path: str):
        self.products = self.load_products(json_file_path)
        self.setup_gemini()
        self.create_vector_store()
        
    def setup_gemini(self):
        api_key = os.getenv('GEMINI_API_KEY')
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-2.0-flash')
    
    def load_products(self, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    
    def create_product_context(self, product: dict) -> str:
        """Create rich context for each product"""
        features = ", ".join(product.get('features', []))
        return f"""
        {product['brand']} {product['name']} - ₹{product['price']}
        • Camera: {product['camera']}MP
        • Battery: {product['battery']}mAh with {product['charging']}W charging
        • RAM: {product['ram']}GB
        • Screen: {product['screen_size']} inches
        • OS: {product['os']}
        • Features: {features}
        """
    
    def create_vector_store(self):
        """Create semantic search index"""
        self.documents = []
        for product in self.products:
            self.documents.append(self.create_product_context(product))
        
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
        self.document_vectors = self.vectorizer.fit_transform(self.documents)
    
    def detect_intent(self, query: str) -> dict:
        """Detect user intent and extract parameters"""
        prompt = f"""
        Analyze this phone query and classify intent with parameters:
        Query: "{query}"
        
        Return JSON with:
        {{
            "intent": "recommendation" | "comparison" | "explanation" | "details",
            "confidence": 0.0-1.0,
            "parameters": {{
                "max_price": number or null,
                "min_price": number or null,
                "brand": string or null,
                "min_camera": number or null,
                "min_battery": number or null, 
                "min_charging": number or null,
                "os": string or null,
                "max_screen_size": number or null,
                "phone_names": array of strings,  // for comparison/details
                "concept": string or null  // for explanation
            }},
            "query_type": "structured" | "semantic"  // numeric vs conceptual
        }}
        
        Intent Guidelines:
        - recommendation: Asking for phone suggestions
        - comparison: Comparing 2+ specific phones (vs, versus, compare)
        - explanation: Explaining technical concepts (what is, explain, vs in concepts)
        - details: Getting info about specific phone (tell me about, details of)
        
        Return only JSON.
        """
        
        try:
            response = self.model.generate_content(prompt)
            if hasattr(response, "text") and response.text:
                raw_text = response.text.strip()
            elif hasattr(response, "candidates") and response.candidates:
                raw_text = response.candidates[0].content.parts[0].text.strip()
            else:
                # print("Gemini returned an unexpected format.")
                return None
            raw_text = re.sub(r"^```json\s*", "", raw_text)
            raw_text = re.sub(r"```$", "", raw_text).strip()
            # print("Gemini raw output:", raw_text)
            try:
                intent_data = json.loads(raw_text)
                return intent_data
            except json.JSONDecodeError:
                print("Response not valid JSON. Returning raw text.")
                return {"intent": "unknown", "confidence": 0.0, "raw_text": raw_text}
                # return json.loads(response.text.strip())
        except Exception as e:
            print(f"Intent detection error: {e}")
            return {"intent": "recommendation", "confidence": 0.7, "parameters": {}}
    
    def structured_search(self, filters: dict) -> list:
        """Exact filtering for numeric queries"""
        results = self.products.copy()
        
        if filters.get('max_price'):
            results = [p for p in results if p['price'] <= filters['max_price']]
        if filters.get('min_price'):
            results = [p for p in results if p['price'] >= filters['min_price']]
        if filters.get('brand'):
            brand = filters['brand'].lower()
            results = [p for p in results if brand in p['brand'].lower()]
        if filters.get('min_camera'):
            results = [p for p in results if p['camera'] >= filters['min_camera']]
        if filters.get('min_battery'):
            results = [p for p in results if p['battery'] >= filters['min_battery']]
        if filters.get('min_charging'):
            results = [p for p in results if p['charging'] >= filters['min_charging']]
        if filters.get('os'):
            os_filter = filters['os'].lower()
            results = [p for p in results if os_filter in p['os'].lower()]
        if filters.get('max_screen_size'):
            results = [p for p in results if p['screen_size'] <= filters['max_screen_size']]
            
        return results
    
    def semantic_search(self, query: str, top_k: int = 5) -> list:
        """Semantic search for conceptual queries"""
        query_vector = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vector, self.document_vectors).flatten()
        
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        results = []
        for idx in top_indices:
            if similarities[idx] > 0.1:
                results.append({
                    'product': self.products[idx],
                    'score': similarities[idx]
                })
        return results
    
    def find_phones_by_name(self, phone_names: list) -> list:
        """Find specific phones by name matching"""
        found_phones = []
        for name in phone_names:
            name_lower = name.lower()
            for product in self.products:
                if name_lower in product['name'].lower() or name_lower in product['brand'].lower():
                    found_phones.append(product)
                    break
        return found_phones
    
    
    def handle_recommendation(self, query: str, intent_data: dict) -> str:
        """Handle phone recommendation requests"""
        print("Handling: RECOMMENDATION")
        
        filters = intent_data['parameters']
        query_type = intent_data.get('query_type', 'semantic')
        
        if query_type == 'structured':
            # Use exact filtering for numeric queries
            results = self.structured_search(filters)
            results = [{'product': p, 'score': 1.0} for p in results]
        else:
            # Use semantic search for conceptual queries
            results = self.semantic_search(query, top_k=8)
            
            # Apply filters to semantic results
            if filters:
                filtered_results = []
                for result in results:
                    product = result['product']
                    if self.apply_filters(product, filters):
                        filtered_results.append(result)
                results = filtered_results
        
        if not results:
            return "No phones found matching your criteria. Try adjusting your requirements."
        
        context = "Available phones matching your query:\n\n"
        for i, result in enumerate(results[:6]):
            product = result['product']
            context += f"{i+1}. {product['name']} - ₹{product['price']:,}\n"
            context += f"   Camera: {product['camera']}MP | Battery: {product['battery']}mAh | "
            context += f"Charging: {product['charging']}W | RAM: {product['ram']}GB\n"
            context += f"   Features: {', '.join(product.get('features', []))}\n\n"
        
        prompt = f"""
        User wants phone recommendations: "{query}"
        
        {context}
        
        Provide a helpful, structured recommendation:
        
        **Best Options for You:**
        [Brief summary of why these match their needs]
        
        **Top Recommendations:**
        [Ranked list with 2-4 best options]
        
        For each phone:
        - **Key Strengths:** [What makes it good for their specific needs]
        - **Spec Highlights:** [Most relevant specs for their query]
        - **Considerations:** [Any trade-offs or things to note]
        
        **Decision Guide:** [Brief comparison of key differences between options]
        
        **Final Advice:** [1-2 sentence summary recommendation]
        
        Be specific and focus on how each phone addresses their stated needs.
        """
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Here are phones matching your criteria:\n{context}"
    
    def handle_comparison(self, query: str, intent_data: dict) -> str:
        """Handle phone comparison requests"""
        print("Handling: COMPARISON")
        
        phone_names = intent_data['parameters'].get('phone_names', [])
        
        if len(phone_names) < 2:
            return "Please specify at least two phones to compare (e.g., 'Compare Phone A vs Phone B')."
        phones_to_compare = self.find_phones_by_name(phone_names)
        
        if len(phones_to_compare) < 2:
            found_names = [p['name'] for p in phones_to_compare]
            return f"Could not find both phones. Found: {', '.join(found_names) if found_names else 'None'}"
        comparison_data = []
        for phone in phones_to_compare:
            comparison_data.append({
                'name': phone['name'],
                'price': phone['price'],
                'camera': phone['camera'],
                'battery': phone['battery'],
                'charging': phone['charging'],
                'ram': phone['ram'],
                'screen_size': phone['screen_size'],
                'os': phone['os'],
                'features': phone.get('features', [])
            })
        
        context = json.dumps(comparison_data, indent=2)
        
        prompt = f"""
        User wants to compare phones: "{query}"
        
        Phones to compare:
        {context}
        
        Provide a detailed comparison:
        
        **Comparison: {phones_to_compare[0]['name']} vs {phones_to_compare[1]['name']}**
        
        **Specifications Comparison:**
        | Feature | {phones_to_compare[0]['name']} | {phones_to_compare[1]['name']} |
        |---------|----------------|----------------|
        | Price | ₹{phones_to_compare[0]['price']:,} | ₹{phones_to_compare[1]['price']:,} |
        | Camera | {phones_to_compare[0]['camera']}MP | {phones_to_compare[1]['camera']}MP |
        | Battery | {phones_to_compare[0]['battery']}mAh | {phones_to_compare[1]['battery']}mAh |
        | Charging | {phones_to_compare[0]['charging']}W | {phones_to_compare[1]['charging']}W |
        | RAM | {phones_to_compare[0]['ram']}GB | {phones_to_compare[1]['ram']}GB |
        | Screen Size | {phones_to_compare[0]['screen_size']}" | {phones_to_compare[1]['screen_size']}" |
        | OS | {phones_to_compare[0]['os']} | {phones_to_compare[1]['os']} |
        
        **Key Differences:**
        [Highlight 3-4 most significant differences]
        
        **Recommendation:**
        - Choose {phones_to_compare[0]['name']} if: [2-3 specific reasons]
        - Choose {phones_to_compare[1]['name']} if: [2-3 specific reasons]
        
        **Verdict:** [Which is better for different use cases]
        """
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            response = f"**Comparison: {phones_to_compare[0]['name']} vs {phones_to_compare[1]['name']}**\n\n"
            for phone in phones_to_compare:
                response += f"**{phone['name']}** - ₹{phone['price']:,}\n"
                response += f"• Camera: {phone['camera']}MP | Battery: {phone['battery']}mAh\n"
                response += f"• Charging: {phone['charging']}W | RAM: {phone['ram']}GB\n\n"
            return response
    
    def handle_explanation(self, query: str, intent_data: dict) -> str:
        """Handle technical explanation requests"""
        print("Handling: EXPLANATION")
        
        concept = intent_data['parameters'].get('concept', '')
        
        prompt = f"""
        User wants explanation: "{query}"
        
        Explain this mobile phone concept in simple, clear terms:
        Concept: {concept or query}
        
        Provide a comprehensive explanation:
        
        **What is {concept or 'this'}?**
        [Simple definition]
        
        **How it works:**
        [Brief technical explanation in layman's terms]
        
        **Why it matters in phones:**
        [Practical benefits and impact on user experience]
        
        **Key Things to Know:**
        [3-5 important points or comparisons]
        
        **Real-world Example:**
        [How this affects phone usage or buying decisions]
        
        Keep it educational but easy to understand for non-technical users.
        Use analogies if helpful.
        """
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"I can explain '{concept or query}'. This feature relates to phone capabilities and user experience. For detailed technical explanations, please rephrase your question."
    
    def handle_details(self, query: str, intent_data: dict) -> str:
        """Handle detailed information requests about specific phones"""
        print("Handling: DETAILS")
        
        phone_names = intent_data['parameters'].get('phone_names', [])
        
        if not phone_names:
            return "Please specify which phone you want details about (e.g., 'Tell me about iPhone 15')."
        
        phones = self.find_phones_by_name(phone_names)
        
        if not phones:
            return f"Sorry, I couldn't find information about '{phone_names[0]}'. Please check the phone name and try again."
        
        phone = phones[0]  
        
        prompt = f"""
        User wants detailed information about: "{query}"
        
        Phone Details:
        {json.dumps(phone, indent=2)}
        
        Provide comprehensive details in this format:
        
        **{phone['name']} - Complete Overview**
        
        **Key Specifications:**
        • **Price:** ₹{phone['price']:,}
        • **Camera:** {phone['camera']}MP rear camera
        • **Battery:** {phone['battery']}mAh with {phone['charging']}W fast charging
        • **Performance:** {phone['ram']}GB RAM, {phone['os']} OS
        • **Display:** {phone['screen_size']} inch screen
        
        **Features & Capabilities:**
        {chr(10).join(['• ' + feature for feature in phone.get('features', [])])}
        
        **Detailed Analysis:**
        [Provide insights about camera quality, performance, battery life, display quality, etc.]
        
        **Best For:**
        [What type of users would benefit most from this phone]
        
        **Considerations:**
        [Any limitations or things to know before buying]
        
        **Verdict:** [Overall assessment of the phone's value and positioning]
        
        Make it informative and helpful for someone considering this phone.
        """
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            # Fallback to basic details
            response = f"**{phone['name']} - Details**\n\n"
            response += f"**Price:** ₹{phone['price']:,}\n"
            response += f"**Camera:** {phone['camera']}MP\n"
            response += f"**Battery:** {phone['battery']}mAh with {phone['charging']}W charging\n"
            response += f"**RAM:** {phone['ram']}GB\n"
            response += f"**Screen:** {phone['screen_size']} inches\n"
            response += f"**OS:** {phone['os']}\n"
            response += f"**Features:** {', '.join(phone.get('features', []))}\n"
            return response
    
    def apply_filters(self, product: dict, filters: dict) -> bool:
        """Helper to apply filters to a product"""
        if filters.get('max_price') and product['price'] > filters['max_price']:
            return False
        if filters.get('min_price') and product['price'] < filters['min_price']:
            return False
        if filters.get('brand') and filters['brand'].lower() not in product['brand'].lower():
            return False
        if filters.get('min_camera') and product['camera'] < filters['min_camera']:
            return False
        if filters.get('min_battery') and product['battery'] < filters['min_battery']:
            return False
        if filters.get('min_charging') and product['charging'] < filters['min_charging']:
            return False
        if filters.get('os') and filters['os'].lower() not in product['os'].lower():
            return False
        if filters.get('max_screen_size') and product['screen_size'] > filters['max_screen_size']:
            return False
        return True
    
    def process_query(self, query: str) -> str:
        """Main method to process any type of query"""
        print(f"\nProcessing: '{query}'")
        
        # Step 1: Detect intent
        intent_data = self.detect_intent(query)
        intent = intent_data.get('intent', 'recommendation')
        confidence = intent_data.get('confidence', 0.7)
        
        print(f"Detected intent: {intent} (confidence: {confidence})")
        
        # Step 2: Route to appropriate handler
        if intent == 'comparison':
            return self.handle_comparison(query, intent_data)
        elif intent == 'explanation':
            return self.handle_explanation(query, intent_data)
        elif intent == 'details':
            return self.handle_details(query, intent_data)
        else:  # recommendation (default)
            return self.handle_recommendation(query, intent_data)

def main():
    system = MultiIntentPhoneSystem("./app/data/products.json")
    
    test_queries = [
        # Recommendation queries
        "Best camera phone under ₹30,000?",
        "Compact Android with good one-hand use",
        "Battery king with fast charging, around ₹25k",
        "Phones with great gaming performance under 35000",
        
        # Comparison queries
        "Compare Nothing Phone 1 vs OnePlus Nord 2T 5G",
        "Pixel 8a vs OnePlus 12R which is better?",
        "Samsung Galaxy S23 vs iPhone 14",
        
        # Explanation queries
        "Explain OIS vs EIS",
        "What is fast charging and how does it work?",
        "Difference between AMOLED and LCD displays",
        
        # Details queries
        "Tell me more about Nothing Phone 1",
        "Details of OnePlus Nord 2T 5G",
        "What are the specs of Samsung Galaxy S23?",
        
        # Mixed/ambiguous
        "Show me Samsung phones only, under ₹25k",
        "I like this phone, tell me more details"
    ]
    
    for query in test_queries:
        print(f"\n{'='*80}")
        print(f"QUERY: {query}")
        print(f"{'='*80}")
        response = system.process_query(query)
        print(response)
        print(f"{'='*80}")

if __name__ == "__main__":
    main()


QUERY: Best camera phone under ₹30,000?

Processing: 'Best camera phone under ₹30,000?'
Detected intent: recommendation (confidence: 0.95)
Handling: RECOMMENDATION
Okay, here's a phone recommendation tailored to your request for the best camera phone under ₹30,000, based on the provided options:

**Best Options for You:**

You're looking for the best camera, so we'll prioritize phones with the highest megapixel counts and those that combine good image sensors with other important features like fast charging and sufficient RAM for smooth operation.

**Top Recommendations:**

1.  **Xiaomi Redmi Note 12 Pro Plus - ₹29,999**
2.  **Realme 10 Pro Plus - ₹24,999**
3.  **OnePlus Nord CE 2 Lite 5G - ₹19,989**

**Xiaomi Redmi Note 12 Pro Plus**

*   **Key Strengths:** This phone boasts the highest megapixel count of the group, giving you potentially more detailed images, especially in good lighting. Super fast charging.
*   **Spec Highlights:**
    *   Camera: 200.0MP
    *   Charging: 120.0W
 